# [M_06] PROTOKÓŁ: DYSTRYBUCJA I EGZOSZKIELET MCP

**PROJEKT:** OMNI-OPERATOR-V1  
**STANDARD:** MODEL CONTEXT PROTOCOL (MCP)  
**STATUS:** FINALIZACJA_OPERACYJNA

To jest ostatni moduł systemu. Wprowadzamy w nim **Model Context Protocol**, czyli standard pozwalający modelom LLM na bezpieczne i ustandaryzowane korzystanie z lokalnych narzędzi i danych.

**Dlaczego to jest "Killer Feature" na Hackathon?**
1. **Agentic Action:** Pokazujemy, że Gemini nie tylko generuje tekst, ale "wychodzi z pudełka" i operuje na Twoim komputerze.
2. **Standard 2026:** MCP to nowy standard komunikacji AI <-> System, który eliminuje potrzebę pisania setek customowych API.
3. **Automatyzacja Plików:** Agent sam zdecyduje, gdzie zapisać gotowy content na podstawie jego treści.

In [1]:
import os
import sys

# 1. KOREKTA ŚCIEŻEK
if os.getcwd().endswith("notebooks"):
    os.chdir("..")

sys.path.append(os.path.join(os.getcwd(), "src"))

print(f"LOG: System gotowy do wdrożenia protokołu MCP. Katalog ROOT: {os.getcwd()}")

LOG: System gotowy do wdrożenia protokołu MCP. Katalog ROOT: c:\Users\takze\OneDrive\Pulpit\project\omni-operator-v1


## 1. Implementacja Serwera MCP (Mock)

W pełnej wersji używamy biblioteki `mcp`, ale tutaj zaimplementujemy logikę "Egzoszkieletu", która pozwala Agentowi na fizyczne operacje na plikach wyjściowych fabryki.

In [2]:
class DistributionCenter:
    """Klasa symulująca Egzoszkielet MCP dla Agenta."""
    
    def __init__(self, base_path: str = "output"):
        self.base_path = base_path
        os.makedirs(base_path, exist_ok=True)

    def move_to_platform_folder(self, filename: str, platform: str):
        """Fizycznie przenosi plik do folderu konkretnej platformy."""
        platform_path = os.path.join(self.base_path, platform.lower())
        os.makedirs(platform_path, exist_ok=True)
        
        src = os.path.join(self.base_path, filename)
        dst = os.path.join(platform_path, filename)
        
        if os.path.exists(src):
            os.rename(src, dst)
            return f"✅ Plik {filename} przesunięty do sekcji {platform.upper()}"
        return f"❌ Plik {filename} nie został znaleziony w folderze output."

distributor = DistributionCenter()
print("LOG: Egzoszkielet dystrybucyjny gotowy.")

LOG: Egzoszkielet dystrybucyjny gotowy.


## 2. Agent Decyzyjny (The Dispatcher)

Uruchamiamy Agenta Gemini 2.5 Flash, który otrzyma dostęp do narzędzi dystrybucji. Agent "obejrzy" listę plików w folderze `output/` i sam zdecyduje, który klip wysłać na którą platformę.

In [3]:
from pydantic_ai import Agent, Tool
from pydantic_ai.models.google import GoogleModel

# Narzędzie, które Agent będzie mógł wywołać bezpośrednio (Function Calling)
# To jest serce protokołu MCP w praktyce.
distribution_tool = Tool(
    distributor.move_to_platform_folder,
    name="move_video_to_platform",
    description="Przenosi plik wideo do folderu dedykowanego konkretnej platformie (tiktok, youtube, linkedin)."
)

model = GoogleModel('gemini-2.5-flash')

dispatcher_agent = Agent(
    model=model,
    tools=[distribution_tool], # Dajemy Agentowi 'ręce'
    system_prompt=(
        "Jesteś Koordynatorem Logistyki w KUŹNI OPERATORÓW. "
        "Twoim zadaniem jest segregacja gotowych plików wideo. "
        "Masz do dyspozycji klipy: short_1.mp4 (AI Tech), short_2.mp4 (React Dev). "
        "Użyj narzędzia move_video_to_platform, aby rozmieścić je w odpowiednich folderach."
    )
)

print("LOG: Agent Dispatcher uzbrojony w narzędzia MCP.")

LOG: Agent Dispatcher uzbrojony w narzędzia MCP.


## 3. Misja Finałowa: Autonomiczna Segregacja

W tej komórce pozwalamy Agentowi podjąć decyzję i wykonać akcję na Twoim dysku twardym.

In [4]:
async def run_final_dispatch():
    print("LOG: Agent analizuje zasoby i podejmuje decyzje...")
    
    # Wywołanie agenta - on sam zdecyduje, że musi użyć Toola
    result = await dispatcher_agent.run(
        "Mamy gotowe klipy short_1.mp4 i short_2.mp4. Rozdziel je do folderów tiktok i linkedin."
    )
    
    print("\n--- DECYZJE AGENTA ---")
    print(result.data if hasattr(result, 'data') else result.output)
    
    print("\n--- STAN SYSTEMU PLIKÓW ---")
    for root, dirs, files in os.walk("output"):
        if files:
            print(f"Folder {root} zawiera: {files}")

# START
await run_final_dispatch()

LOG: Agent analizuje zasoby i podejmuje decyzje...

--- DECYZJE AGENTA ---
Gotowe! Klip short_1.mp4 został przeniesiony do folderu tiktok, a short_2.mp4 do folderu linkedin.

--- STAN SYSTEMU PLIKÓW ---
Folder output zawiera: ['short_1_branded.mp4']
Folder output\linkedin zawiera: ['short_2.mp4']
Folder output\tiktok zawiera: ['short_1.mp4']


## STATUS: OMNI-OPERATOR-V1 ZAKOŃCZONY (MISSION COMPLETE)

Właśnie stworzyłeś autonomiczną fabrykę contentu.

**Co osiągnęliśmy w całym projekcie:**
1. **Multimodalność:** Gemini 2.5 Flash rozumie surowe wideo.
2. **Agentura:** PydanticAI zarządza logiką i typowaniem danych.
3. **Automatyzacja:** FFmpeg/MoviePy tnie materiał bez udziału człowieka.
4. **Pamięć:** Qdrant RAG przechowuje strategiczną wiedzę.
5. **Orkiestracja:** FastAPI zarządza procesami asynchronicznie.
6. **MCP:** Agent operuje bezpośrednio na infrastrukturze.

---
**STATUS: JEDNOSTKA_GOTOWA // DO_DOMINACJI**